In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler

Code for running PLS using 1 up to n_comp components and returning the number of components which yielded the highest CV $R^{2}$:

In [ ]:
def optimise_pls_cv(X, y, n_comp, cv_f, plot_components=True):
 
    '''Run PLS including a variable number of components, up to n_comp,
       and calculate MSE '''
 
    r2, mse = [], []
    if X.shape[1] < n_comp: # if features are less than maximum number of components
        if X.shape[1] == 1: # when 1 feature left, use 1 component 
            component = [1]
        else:
            component = np.arange(1, X.shape[1]) # max components are 1 less than total features
    else:
        component = np.arange(1, n_comp)
 
    for i in component:
        pls = PLSRegression(n_components=i)
        # Cross-validation
        y_cv = cross_val_predict(pls, X, y, cv=cv_f)
        mse.append(mean_squared_error(y, y_cv))
        r2.append(r2_score(y, y_cv))
 
    # Calculate and print the position of minimum in MSE
    if not mse:
        msemin = 0
    else:
        msemin = np.argmin(mse)
    if not r2:
        r2max = r2_score(y, y_cv)
    else:
        r2max = np.argmax(r2)
 
    if plot_components is True:
        with plt.style.context(('ggplot')):
            plt.plot(component, np.array(r2), '-v', color = 'blue', mfc='blue')
            plt.plot(component[r2max], np.array(r2)[r2max], 'P', ms=10, mfc='red')
            plt.xlabel('Number of PLS components')
            plt.ylabel('R-Sq')
            plt.title('PLS')
            plt.xlim(left=-1)
 
        plt.show()
 
    return r2max+1, np.max(r2), np.min(mse)

Code for running PLS using optimal number of components and RFE to identify the best-performing CSC-NPP model:

In [ ]:
def pls_rfe_cv(X_df, X, y, max_comp, cv_fold):
    r2_cv, mse_fit = np.zeros(X.shape[1]), np.zeros(X.shape[1]) # store CV r2 and mse from max number of features to 1
    r2_fit, mse_cv = np.zeros(X.shape[1]), np.zeros(X.shape[1]) # store r2 and mse of fitting data from max number of features to 1
    metrics_det = list(X_df.columns[2:]) # initiate with all metric names
    metrics_sub = np.empty(X.shape[1], dtype=object) # store list of included metrics after each elimination
    metrics_sub[-1] = metrics_det # append full set of metrics as last element
    coeffs = np.empty(X.shape[1], dtype=object) # store coefficients of included metrics after each elimination
    r2_best, n_comp_best = 0, 0 # store r2 score and number of components selected in best model

    # Recursively eliminate least important features until 1 is left
    for i in reversed(range(0, len(r2_cv))):
        # Find optimal number of components
        n_comps, r2_max, mse_min = optimise_pls_cv(X, y, max_comp, cv_fold, plot_components=False)
        print(f"Number of components selected with {i+1} features:", n_comps)
        if r2_max > r2_best: # Store number of components of best model (highest CV r2 score)
            r2_best = r2_max
            n_comp_best = n_comps
        r2_cv[i] = r2_max # store cross-validation max r2
        mse_cv[i] = mse_min # store cross-validation min mse
        # Fit the model to the training data
        pls = PLSRegression(n_components=n_comps)
        pls.fit(X, y)
        coeffs[i] = list(pls.coef_.reshape(-1)) # Store included coefficients after elimination
        y_est = pls.predict(X)
        r2 = r2_score(y, y_est)
        mse = mean_squared_error(y, y_est)
        r2_fit[i] = r2 # store r2 score with training set
        mse_fit[i] = mse # store mse with training set
        # Eliminate least important feature (lowest absolute coefficient)
        el_ind = np.argmin(np.absolute(pls.coef_)) # index of lowest coefficient
        metrics_det = np.delete(metrics_det, el_ind) # delete least important metric name 
        if i>0:
            metrics_sub[i-1] = metrics_det # Store included metrics after elimination
        X = np.delete(X, el_ind, axis=1) # delete least important column for next iteration

    # Return list of CV values, Fit values, included metrics, included coefficients, and # of components of best model
    return r2_cv, mse_cv, r2_fit, mse_fit, metrics_sub, coeffs, n_comp_best